<a href="https://colab.research.google.com/github/dr-kinder/playground/blob/dev/magnetic-field-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Finite Element Method

To solve hard electrodynamics problems, we are going to use the finite element method.  This tutorial will help you get familiar with the process.

## Acknowledgements

The installation commands below come from the [FEM on CoLab](https://fem-on-colab.github.io/index.html) project.

This tutorial was adapted from "Solving the Poisson Equation" in [The FEniCSx Tutorial](https://jorgensd.github.io/dolfinx-tutorial/chapter1/fundamentals.html).

Thanks to all of the contributors to these projects!

## FEniCSx, GMSH, and Multiphenicsx

The finite element method involves a lot of numerical methods that would be difficult for us to build from scratch.  Instead, we will draw from a few open-source projects with state-of-the-art implementations.

- [**FEniCSx**](https://fenicsproject.org/) is the workhorse of our finite element calculations.
- [**Gmsh**](https://gmsh.info/) is a library we will use to define the geometry of the systems under study.  It is used in some computer-aided drafting (CAD) applications, too.
- [**Multiphenicsx**](https://github.com/multiphenics/multiphenicsx) is a package built on top of FEniCSx with some nice functions for plotting finite element meshes, scalar functions, and vector fields.

The code in the next three cells will install these three packages within your CoLab session.

The `try ... except` lines below will attempt to import the libraries we need.  If they are not found, they will download and install the libraries.  These commands have been tested on CoLab.  They probably won't work properly elsewhere.

It takes a little while to download and install the libraries, so hit "Play" on the next three cells, relax and grab a cup of coffee, and finish checking your email.  Then get ready to solve some boundary value problems!

In [ ]:
try:
    # Import gmsh library for generating meshes.
    import gmsh
except ImportError:
    # If it is not available, install it.  Then import it.
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

In [ ]:
try:
    # Import FEniCSx libraries for finite element analysis.
    import dolfinx
except ImportError:
    # If they are not found, install them.  Then import them.
    !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
try:
    # Import multiphenicsx, mainly for plotting.
    import multiphenicsx
except ImportError:
    # If they are not found, install them.
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@8b97b4e"
    import multiphenicsx

Everything we need should be installed now!

If you "Restart runtime" from the "Runtime" menu, all of your data will be reset, but the packages will remain installed.

Let's load the packages we need, and get started!

In [ ]:
# Everything should be installed now.
# Import the rest of what we need.

import dolfinx.fem
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import petsc4py.PETSc
import ufl
import multiphenicsx.fem
import multiphenicsx.io

In [ ]:
# Create a rectangle.
# Locate the center.
x0 = 0
y0 = 0
z0 = 0

# Define the shape.
L = 2
W = 2

# Locate the corner.
x0 = x0 - L/2
y0 = y0 - W/2

# Tell the modeling program how many dimensions we are using.
dim = 2

# Grid size parameter.  Make it smaller for higher resolution.
delta = 0.1

# Create a model.
gmsh.initialize()
gmsh.model.add("mesh")

# Define points: corners of the rectangle.
p0 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
p1 = gmsh.model.geo.addPoint(x0+L, y0, z0, delta)
p2 = gmsh.model.geo.addPoint(x0+L, y0+W,z0, delta)
p3 = gmsh.model.geo.addPoint(x0, y0+W, z0, delta)

# Define the perimeter of the rectangle as a loop.
l0 = gmsh.model.geo.addLine(p0, p1)
l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p0)
loop = gmsh.model.geo.addCurveLoop([l0,l1,l2,l3])

# Define the interior of the rectangle as a surface.
rectangle = gmsh.model.geo.addPlaneSurface([loop])


# Define the center of the circle.
x0 = 0
y0 = 0
z0 = 0

# Define the radius of the circle.
r0 = 3

# Define points: center of circle and two points on opposite sides.
p0 = gmsh.model.geo.addPoint(x0,y0, z0, delta)
p1 = gmsh.model.geo.addPoint(x0, y0-r0, z0, delta)
p2 = gmsh.model.geo.addPoint(x0, y0+r0,z0, delta)

# Define two semicircular arcs that will be joined into a circle.
c0 = gmsh.model.geo.addCircleArc(p1, p0, p2)
c1 = gmsh.model.geo.addCircleArc(p2, p0, p1)
loop = gmsh.model.geo.addCurveLoop([c0,c1])

## Alternate geometry: semicircle
# l0 = gmsh.model.geo.addLine(p2, p1)
# loop = gmsh.model.geo.addCurveLoop([c0, l0])

disk = gmsh.model.geo.addPlaneSurface([loop])

# Update the model with all of the features we add.
gmsh.model.geo.synchronize()

# Some geometric objects were only used to define others.
# Identify the physical objects.

# Add each edge separately, so it is a unique object.
# This will allow us to set the boundary conditions separately.
gmsh.model.addPhysicalGroup(1, [l0], 1)
gmsh.model.addPhysicalGroup(1, [l1], 2)
gmsh.model.addPhysicalGroup(1, [l2], 3)
gmsh.model.addPhysicalGroup(1, [l3], 4)

# Define the interior of the rectangle as our domain.
gmsh.model.addPhysicalGroup(2, [rectangle], 1)

# Define the interior of the disk as our domain.
gmsh.model.addPhysicalGroup(1, [c0,c1], 5)
gmsh.model.addPhysicalGroup(2, [disk], 2)


# Create a mesh for this system.
gmsh.model.mesh.generate(dim)

# Bring the mesh into FEniCSx.
mesh, subdomains, boundaries = dolfinx.io.gmshio.model_to_mesh(
    gmsh.model, comm=mpi4py.MPI.COMM_WORLD, rank=0, gdim=2)

# Close the mesh generating program.
gmsh.finalize()

In [ ]:
# Get separate identifiers for the four walls.
wall_1 = boundaries.indices[boundaries.values == 1]
wall_2 = boundaries.indices[boundaries.values == 2]
wall_3 = boundaries.indices[boundaries.values == 3]
wall_4 = boundaries.indices[boundaries.values == 4]

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Plot the subdomains that FEniCSx has identified.
# There should only be one for this model.
multiphenicsx.io.plot_mesh_tags(subdomains)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Inspect the boundaries of the elements and the system.
multiphenicsx.io.plot_mesh_tags(boundaries)

# Note that each wall is a different color,
# indicating it is a separate "physical object".

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Change the last function argument to see the individual walls.
multiphenicsx.io.plot_mesh_entities(mesh, mesh.topology.dim - 1, wall_1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
## Set up the finite element problem.

# Define trial and test functions.
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))

# Define the trial and test functions.
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

# Create a function to store the solution.
phi = dolfinx.fem.Function(V)

# Identify the domain (all the points inside the boundary).
Omega = subdomains.indices[subdomains.values == 1]
dOmega = boundaries.indices[boundaries.values != 0]

# Use these objects to tell FEniCSx where the boundary is.
boundary_elements = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, dOmega)

# Now introduce the boundary condition: constant potential on the boundary.
phi0 = 0
Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(phi0))
bc = dolfinx.fem.dirichletbc(Phi0, boundary_elements, V)

#####

# Identify the boundaries for FEniCSx.
# Use these objects to tell FEniCSx where the boundary is.
dOmega_1 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_1)
dOmega_2 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_2)
dOmega_3 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_3)
dOmega_4 = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, wall_4)

# Now introduce the boundary condition: constant potential on the boundary.
phi0 = 0.0
phi1 = 2.0
Phi0 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(phi0))
Phi1 = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(phi1))

# Fix the potential on the walls.
bc_1 = dolfinx.fem.dirichletbc(Phi1, dOmega_1, V)
bc_2 = dolfinx.fem.dirichletbc(Phi0, dOmega_2, V)
bc_3 = dolfinx.fem.dirichletbc(Phi0, dOmega_3, V)
bc_4 = dolfinx.fem.dirichletbc(Phi0, dOmega_4, V)

# Combine all boundary conditions into a list for FEniCSx.
bc = [bc_1, bc_2, bc_3, bc_4]

# This is the FEM version of the Laplacian.
# It is the left-hand side of Poisson's equation.
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx

# This the right-hand side of Poisson's equation.
# We need to create a FEniCSx-friendly version of 0.
Zero = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0.0))
One = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(1.0))

L = Zero * v * ufl.dx

# Put it all together for FEniCSx.
# problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[bc], u=phi, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=bc, u=phi)

# Now, solve it!
problem.solve()

# Tie up some loose ends.
phi.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
# Plot the solution.
multiphenicsx.io.plot_scalar_field(phi, "Potential", 1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
# Plot the solution.
multiphenicsx.io.plot_scalar_field(phi, "Potential", 1)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

## Create a Model

We are going to solve the Poisson equation for charged wire inside a grounded, conducting cylinder.  We will assume the cylinder and wire are long enough that we can ignore any variation along the axis of the cylinder and wire, and we focus our attention on a 2D cross section.

Thus, we need a circular disk as our space, and we need to identify the points on the boundary, so we can set them equal to zero.

The Gmsh package will allow us to do this.

Don't just run the following code.  Read through it and try to figure out how it works.

We should have a mesh now: a collection of nodes and elements.  Let's see what we've created.

Gmsh has just created this model for us.  Describe what you see.

- Did we make the correct shape?
- What kind of "mesh" did Gmsh create?
- Does the mesh seem reasonable?

***Replace with your response.***

## Finite Element Method

We now have a physical system and a grid to work with.  The next step is to define our problem in such a way that FEniCSx can solve it.

The first step is to define a set of functions on our grid.*italicized text*

### Charge Density

Next, we will define the charge density of the wire.  We use a Gaussian distribution (bell curve) here.

Define the charge density.  We will use a Gaussian charge distribution.  You can adjust the center and the spread of the distribution.

We can plot the charge density.  First, we have to interpolate the symbolic expression onto our grid.

### Electric Field


The electric field is the (negative) gradient of the potential, so we can compute and visualize this, too!

Make another prediction first, though.

What do you expect the electric field to look like?

***Replace with your prediction.***

In [ ]:
# Define a set of elements for a vector field.
W = dolfinx.fem.VectorFunctionSpace(mesh, ("Lagrange", 2))
E = dolfinx.fem.Function(W)

# Compute the gradient as a symbolic expression, then interpolate it onto the mesh.
expr = dolfinx.fem.Expression(ufl.as_vector((-phi.dx(0), -phi.dx(1))), W.element.interpolation_points())
E.interpolate(expr)

The `glyph_factor` controls the appearance of the arrows, similar to the `warp_factor` for a scalar field.  Adjust it to make the plot easier to interpret.

In [ ]:
# Use multiphenics to plot the vector field.
multiphenicsx.io.plot_vector_field(E,name="Electric Field", glyph_factor=1e-2)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Describe what you see.
 
- What does the electric field look like?
- Did you predict the main features of the field?  If not, what surprised you?

***Replace with your response.***

## Reflection

Summarize what you've learned in working through this notebook.

***Replace with your response.***

## Challenge

Make a copy of this notebook in your GitHub repository.  Then, modify the code and explore the results.

Some things to try ...
- Move the charge around inside the disk.  How does its location affect the potential and electric field?
- Change the size of the disk.  How do you think the potential and fields will change?
- Change the spread of the charge distribution.  How does this affect the potential and electric field?
- Change the fixed potential on the boundary.  How does this affect the potential and electric field?
- Put two charges inside the system — one positive and one negative.  How does that affect the resulting potential and field?

Try any of these, or your own ideas.

In [ ]:
import gmsh
import numpy as np
from mpi4py import MPI

rank = MPI.COMM_WORLD.rank

gmsh.initialize()
r = 0.1   # Radius of copper wires
R = 5     # Radius of domain
a = 1     # Radius of inner iron cylinder
b = 1.2   # Radius of outer iron cylinder
N = 3     # Number of windings
c_1 = 0.8 # Radius of inner copper wires
c_2 = 1.4 # Radius of outer copper wires
gdim = 2  # Geometric dimension of the mesh
model_rank = 0
mesh_comm = MPI.COMM_WORLD
if mesh_comm.rank == model_rank:

    # Define geometry for iron cylinder
    outer_iron = gmsh.model.occ.addCircle(0, 0, 0, b)
    inner_iron = gmsh.model.occ.addCircle(0, 0, 0, a)
    gmsh.model.occ.addCurveLoop([outer_iron], 5)
    gmsh.model.occ.addCurveLoop([inner_iron], 6)
    iron = gmsh.model.occ.addPlaneSurface([5, 6])
    gmsh.model.occ.synchronize()

    # Define geometry for background
    background = gmsh.model.occ.addDisk(0, 0, 0, R, R)
    gmsh.model.occ.synchronize()

    # Define the copper-wires inside iron cylinder
    angles_N = [i*2*np.pi/N for i in range(N)]
    wires_N = [(2, gmsh.model.occ.addDisk(c_1*np.cos(v), c_1*np.sin(v), 0, r, r)) for v in angles_N]

    # Define the copper-wires outside the iron cylinder
    angles_S = [(i+0.5)*2*np.pi/N for i in range(N)]
    wires_S = [(2, gmsh.model.occ.addDisk(c_2*np.cos(v), c_2*np.sin(v), 0, r, r)) for v in angles_S]
    gmsh.model.occ.synchronize()
    # Resolve all boundaries of the different wires in the background domain
    all_surfaces = [(2, iron)]
    all_surfaces.extend(wires_S)
    all_surfaces.extend(wires_N)
    whole_domain = gmsh.model.occ.fragment([(2, background)], all_surfaces)
    gmsh.model.occ.synchronize()
    # Create physical markers for the different wires.
    # We use the following markers:
    # - Vacuum: 0
    # - Iron cylinder: 1
    # - Inner copper wires: $[2,3,\dots,N+1]$
    # - Outer copper wires: $[N+2,\dots, 2\cdot N+1]$
    inner_tag = 2
    outer_tag = 2 + N
    background_surfaces = []
    other_surfaces = []
    for domain in whole_domain[0]:
        com = gmsh.model.occ.getCenterOfMass(domain[0], domain[1])
        mass = gmsh.model.occ.getMass(domain[0], domain[1])
        # Identify iron circle by its mass
        if np.isclose(mass, np.pi*(b**2-a**2)):
            gmsh.model.addPhysicalGroup(domain[0], [domain[1]], tag=1)
            other_surfaces.append(domain)
        # Identify the background circle by its center of mass
        elif np.allclose(com, [0, 0, 0]):
            background_surfaces.append(domain[1])

        # Identify the inner circles by their center of mass
        elif np.isclose(np.linalg.norm(com), c_1):
            gmsh.model.addPhysicalGroup(domain[0], [domain[1]], inner_tag)
            inner_tag +=1
            other_surfaces.append(domain)
        # Identify the outer circles by their center of mass
        elif np.isclose(np.linalg.norm(com), c_2):
            gmsh.model.addPhysicalGroup(domain[0], [domain[1]], outer_tag)
            outer_tag +=1
            other_surfaces.append(domain)
    # Add marker for the vacuum
    gmsh.model.addPhysicalGroup(2, background_surfaces, tag=0)
    # Create mesh resolution that is fine around the wires and
    # iron cylinder, coarser the further away you get
    gmsh.model.mesh.field.add("Distance", 1)
    edges = gmsh.model.getBoundary(other_surfaces, oriented=False)
    gmsh.model.mesh.field.setNumbers(1, "EdgesList", [e[1] for e in edges])
    gmsh.model.mesh.field.add("Threshold", 2)
    gmsh.model.mesh.field.setNumber(2, "IField", 1)
    gmsh.model.mesh.field.setNumber(2, "LcMin", r / 2)
    gmsh.model.mesh.field.setNumber(2, "LcMax", 5 * r)
    gmsh.model.mesh.field.setNumber(2, "DistMin", 2 * r)
    gmsh.model.mesh.field.setNumber(2, "DistMax", 4 * r)
    gmsh.model.mesh.field.setAsBackgroundMesh(2)
    # Generate mesh
    gmsh.option.setNumber("Mesh.Algorithm", 7)
    gmsh.model.mesh.generate(gdim)

In [ ]:
from dolfinx.io.gmshio import model_to_mesh
mesh, ct, _ = model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=2)
gmsh.finalize()

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_mesh(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
from dolfinx.fem import (dirichletbc, Expression, Function, FunctionSpace, 
                         VectorFunctionSpace, locate_dofs_topological)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.mesh import locate_entities_boundary
from ufl import TestFunction, TrialFunction, as_vector, dot, dx, grad, inner
from petsc4py.PETSc import ScalarType

Q = FunctionSpace(mesh, ("DG", 0))
material_tags = np.unique(ct.values)
mu = Function(Q)
J = Function(Q)
# As we only set some values in J, initialize all as 0
J.x.array[:] = 0
for tag in material_tags:
    cells = ct.find(tag)
    # Set values for mu
    if tag == 0:
        mu_ = 1 # Vacuum
    elif tag == 1:
        mu_ = 1 # Iron (This should really be 6.3e-3)
    else:
        mu_ = 10 # Copper
    mu.x.array[cells] = np.full_like(cells, mu_, dtype=ScalarType)
    if tag in range(2, 2+N):
        J.x.array[cells] = np.full_like(cells, 1, dtype=ScalarType)
    elif tag in range(2+N, 2*N + 2):
        J.x.array[cells] = np.full_like(cells, -1, dtype=ScalarType)

In [ ]:
V = FunctionSpace(mesh, ("Lagrange", 2))
tdim = mesh.topology.dim
facets = locate_entities_boundary(mesh, tdim-1, lambda x: np.full(x.shape[1], True))
dofs = locate_dofs_topological(V, tdim-1, facets)
bc = dirichletbc(ScalarType(0), dofs, V)

u = TrialFunction(V)
v = TestFunction(V)
a = (1 / mu) * dot(grad(u), grad(v)) * dx
L = J * v * dx

In [ ]:
A_z = Function(V)
problem = LinearProblem(a, L, u=A_z, bcs=[bc])
problem.solve()

Coefficient(FunctionSpace(Mesh(VectorElement(Basix element (P, triangle, 1, equispaced, unset, False), 2), 7), Basix element (P, triangle, 2, gll_warped, unset, False)), 71)

In [ ]:
W = VectorFunctionSpace(mesh, ("CG", 2))
B = Function(W)
B_expr = Expression(as_vector((A_z.dx(1), -A_z.dx(0))), W.element.interpolation_points())
B.interpolate(B_expr)

In [ ]:
# Plot the entire mesh.
multiphenicsx.io.plot_scalar_field(A_z,"Vector Potential", warp_factor=2)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
multiphenicsx.io.plot_vector_field(B,"Magnetic Field", glyph_factor=0.5)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…